In [70]:
from importlib import reload
import data_loader as dl

You can reload the data_loader.py without restarting the kernel using `reload(data_loader)`

In [71]:
reload(dl)

<module 'data_loader' from 'D:\\College\\Level 4\\Fall Semester\\CSE 472 - Data Mining (NLP)\\Labs\\nlp-assignment1\\src\\data_loader.py'>

In [72]:
data_loader = dl.DataLoader()  # using the default file paths and columns to select.
data_loader.load_preprocess_data()
data_loader.describe_data()

+----------------------+-------------+----------------+-------------------------+---------------------------+
| DataFrame            |   # of Rows |   # of Columns | Column Names            | Label Statistics          |
+======================+=============+================+=========================+===========================+
| Training Dataframe   |        8544 |              3 | sentence, label, tokens | label                     |
|                      |             |                |                         | 3    2322                 |
|                      |             |                |                         | 1    2218                 |
|                      |             |                |                         | 2    1624                 |
|                      |             |                |                         | 4    1288                 |
|                      |             |                |                         | 0    1092                 |
|         

Use `data_loader.get_data()` to get the 3 dataframes

In [73]:
train_df, test_df, validation_df = data_loader.get_data()